# UGCrawler 0.8 alpha

This Notebook contains code fragments to crawl ultimate-guitar.com. It is far from finished but all the major components are there. 

Since it is not feasible for me to generate a complete rip, i am sharing this with all the guitarists out there who are pissed off by UG for any of the million reasons given. 

If you would like to contribute, by doing part of the crawl, of implementin a better selection logic, or in any other way, feel free to do so.

In [ ]:
from lxml import etree
import lxml.html
import string
import urllib.request
from time import sleep
import sys

In [ ]:
chars = ['0-9'] + list(string.ascii_lowercase)

In [ ]:
def get_band_urls(char):    
    base_url = 'https://www.ultimate-guitar.com/bands/{}.htm'
    
    htmlparser = etree.HTMLParser()
    
    band_dict = {}
    
    cnt = 0
    while True:
        sleep(0.2)
        
        postfix = '' if cnt == 0 else cnt
        response = urllib.request.urlopen(base_url.format(char + str(postfix)))
        tree = etree.parse(response, htmlparser)

        # collect song links
        tab_table = tree.xpath('//*[contains(@class, "b3")]/../table[2]/tr/td[2]/a')
        
        # empty means no more pages
        if not tab_table:
            break
            
        for child in tab_table:
            band_dict[child.text[:-5]] = child.attrib['href']
        cnt += 1
        
    return band_dict

In [ ]:
def get_artist_links(url):
    base_url = 'https://www.ultimate-guitar.com'

    url = base_url + url[:-4] + '{}' + url[-4:]
    
    htmlparser = etree.HTMLParser()
    
    all_tabs_chords = []
    
    cnt = 0
    tries = 0
    while True:
        sleep(1)
        
        # page 0 exists and is pretty much page 1 without the album shit
        if cnt == 1:
            cnt = 2
            continue
            
        try:
            response = urllib.request.urlopen(url.format(cnt))
        except urllib.error.HTTPError as e:
            if e.code == 404:
                # end of tab sites
                break
            else:
                # retry
                if tries > 5:
                    cnt += 1
                    tries = 0
                else:
                    tries += 1
                continue
                    
        tries = 0
        
        tree = etree.parse(response, htmlparser)
        
        # collect entries
        tab_table = tree.xpath("//td/b[./text()='Tab']/../..")
        chords_table = tree.xpath("//td/b[./text()='Chords']/../..")

        for song in tab_table:
            name = song.xpath('./td[1]/a[1]')[0].text    
            link = song.xpath('./td[1]/a[1]')[0].attrib['href']

            if not song.xpath('./td[2]/span[1]'):
                rating = 0
            else:
                rating = song.xpath('./td[2]/span[1]')[0].attrib['title']

            all_tabs_chords.append(
                {
                    'name': name,
                    'link': link,
                    'rating': rating,
                    'type': 'Tab'
                }
            )
            
        for song in chords_table:
            name = song.xpath('./td[1]/a[1]')[0].text    
            link = song.xpath('./td[1]/a[1]')[0].attrib['href']

            if not song.xpath('./td[2]/span[1]'):
                rating = 0
            else:
                rating = song.xpath('./td[2]/span[1]')[0].attrib['title']

            all_tabs_chords.append(
                {
                    'name': name,
                    'link': link,
                    'rating': rating,
                    'type': 'Chords'
                }
            )        
        cnt += 1
        
    return all_tabs_chords

In [ ]:
def get_print_version(url):
    tries = 0
    while tries < 5:
        try:
            response = urllib.request.urlopen(url)
        except urllib.error.HTTPError as e:
            if e.code == 404:
                return None
            else:
                tries += 1
                continue
                
        htmlparser = etree.HTMLParser()
        tree = etree.parse(response, htmlparser)
        
        # get link
        link = tree.xpath("//*[@id='print_link']")[0].attrib['href']
        response = urllib.request.urlopen('https://tabs.ultimate-guitar.com{}'.format(link))
        
        ht = lxml.html.parse(response)
        return ht.xpath("//div[contains(@class, 'tb_ct')]")[0].text_content()

### Here are some examples of how the functions work, and what they return, should be self explanatory

In [ ]:
band_urls = get_band_urls('w')
band_urls

In [ ]:
artist_links = get_artist_links('/tabs/we_are_scientists_tabs.htm')
artist_links

In [ ]:
tab = get_print_version('https://tabs.ultimate-guitar.com/w/we_are_scientists/lethal_enforcer_acoustic_crd.htm')
print(tab)

### This is pretty much Pseudo Code. Nothing works yet.

This is the outline of the process of crawling the ENTIRE site. UG WILL BLOCK YOU IF YOU TRY THIS.

This needs some logic to filter out the top rated tabs/chords for everything.

In [ ]:
all_the_stuff = {}

for c in chars:
    # rip the band urls
    band_urls = get_band_urls(c)
    
    # rip the tabs for each band
    for artist, url in band_urls.items():
        
        all_the_stuff[artist] = {}
        
        # rip
        print('Crawling {}...'.format(artist))
        song_links = get_artist_links(url)
        
        # process
        for song_dict in song_links:
            entry_name = '{} | {}'.format(song_dict['name'], song_dict['type'])
            all_the_stuff[artist][entry_name] = get_print_version(song_dict['link'])

        

    sleep(5)